  # Transcript Agent

*Visto a dificuldade enfrentada pelo grupo do short talks em legendar os videos e assim atrasando a publicação das entrevistas e engajamento da iniciativa, foi pensando como uma forma de facilitar a maneira de obter esse roteiro de maneira simples podendo assim editar o video com mais facilidade sem dependências externas, de inicio o agente apenas transcreve os videos/audios das entrevistas feita pleo time do ECAC short talks.*
<br>
<br>
<br>
Antes de começar, precisamos instalar todas as dependências necessárias para este agente.

In [ ]:
!pip uninstall -y whisper
!pip install git+https://github.com/openai/whisper.git
!pip install torchaudio --upgrade
!pip install moviepy
!pip install langchain openai

Após isso iremos importar as dependencias e em seguida montar o drive aonde estara o video para trancrição

In [ ]:
from moviepy.editor import VideoFileClip
import whisper
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive')

video_file = '/content/drive/MyDrive/Your-folder/your-video'
if not os.path.exists(video_file):
    raise FileNotFoundError(f"Not possible to find the video: {video_file}")

Configurando nossa Key

In [ ]:
os.environ["OPENAI_API_KEY"] = input("Digite sua OpenAI API Key: ")
llm = OpenAI(temperature=0.5, max_tokens=700)

Agora chamamos as funções principais

In [ ]:
def extrair_audio(video_path):
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    audio_path = f"/content/{base_name}.mp3"
    clip = VideoFileClip(video_path)
    clip.audio.write_audiofile(audio_path, codec="libmp3lame", verbose=False, logger=None)
    clip.close()
    return audio_path

def transcrever_audio(audio_path, model_name="base"):
    model = whisper.load_model(model_name)
    result = model.transcribe(audio_path, language="pt", fp16=False)
    return result["text"]

Chamando as funções e salvando o roterio direto no drive

In [ ]:
audio_file = extrair_audio(video_file)

roteiro = transcrever_audio(audio_file)
print("----- Roteiro do Vídeo -----\n")
print(roteiro)

roteiro_path = '/content/drive/MyDrive/ECAC ST/roteiro.txt'
with open(roteiro_path, "w", encoding="utf-8") as f:
    f.write(roteiro)